In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import myFunctions as my
%load_ext autoreload
%autoreload 2

## Load the training data 
into feature matrix X, class labels y, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, X, ids = load_csv_data(DATA_TRAIN_PATH)

## Data cleaning
Here we deal with problematic values.

In [3]:
from helpers import *

def standardize_badFeatures(X):
    
    # Function that calculate the mean and std of bad features without elements equal to -999
    # Then, it remplaces -999 values by zeros, zeros won't influence the train of the model... 
    mean_x = np.zeros((X.shape[1],))
    std_x = np.zeros((X.shape[1],))
    for d in range(X.shape[1]):
        idx = np.where(X[:,d] == -999)[0]
        mean_x[d] = np.mean(np.delete(X[:,d], (idx)))
        std_x[d] = np.std(np.delete(X[:,d], (idx)))
        X[:,d] = (X[:,d]-mean_x[d])/std_x[d]
        X[idx,d] = 0
    return X, mean_x, std_x


def clean_data(X):

    # find indices of features that have at least one value -999, we call them "bad" features
    idx_badFeatures = []
    for d in range(X.shape[1]):
        if sum(X[:,d] == -999) > 0:
            idx_badFeatures.append(d)

    # separate "good" and "bad" features
    X_badFeatures = X[:,idx_badFeatures]
    X_goodFeatures = np.delete(X,(idx_badFeatures), axis=1)

    # Standardize it differently (see : standardize_badFeatures(X))
    tX, mean_x, std_x = standardize(X_goodFeatures)
    tX2, mean_x2, std_x2 = standardize_badFeatures(X_badFeatures)

    # comment the 3 next lines if you want to work only with "good" features
    tX = np.hstack((tX, tX2))
    mean_x = np.hstack((mean_x, mean_x2))
    std_x = np.hstack((std_x, std_x2))
    
    return tX, mean_x, std_x

# Now tX already has ones in the first column...
tX, mean_x, std_x = clean_data(X)

In [ ]:
# You can work with less samples, it saves time.
#print(tX.shape)
#N = 100000
#tX = tX[0:N]
#y = y[0:N]
#print(tX.shape)
# indices of each class
idx_red = np.where(y == 1)[0]
idx_blue = np.where(y == -1)[0]

#for d in range(1,tX.shape[1]):
#    f, axarr = plt.subplots(2, sharex=True)
#    axarr[0].hist(tX[idx_red,d], 50, normed=1, facecolor='red', alpha=0.5)
#    axarr[1].hist(tX[idx_blue,d], 50, normed=1, facecolor='blue', alpha=0.5)
#    plt.show()

# look for outliers (> 40*std)
#outliers = np.where(tX > 40)
# here we see that our single outlier is in fact a higgs boson event...
# should we keep it ???
#print(outliers[0], y[outliers[0]])
#plt.boxplot(tX)
#np.sum(y == 1)/len(y), np.sum(y == -1)/len(y)


#Is there samples that are identical ?
#ncols = tX.shape[1]
#dtype = tX.dtype.descr * ncols
#struct = tX.view(dtype)

#uniq, idx = np.unique(struct, return_index=True)
#tX = uniq.view(tX.dtype).reshape(-1, ncols)

#print(tX.shape)

# Method 4 : Ridge regression

In [ ]:
from plots import bias_variance_decomposition_visualization

def cross_validation(y, x, k_indices, k, lamb):
    """return the loss of ridge regression."""
    
    # get k'th subgroup in test, others in train: TODO
    tx_tr = x[np.delete(k_indices, (k), axis=0).flatten()]
    y_tr = y[np.delete(k_indices, (k), axis=0).flatten()]
    tx_te = x[k_indices[k]]
    y_te = y[k_indices[k]]
    
    # regression/classification method
    #w = my.least_squares(y_tr, tx_tr)
    w = my.ridge_regression(y_tr, tx_tr, lamb)
    
    # calculate the loss for train and test data: TODO
    #loss_tr = my.compute_loss(y_tr, tx_tr, w)
    #loss_te = my.compute_loss(y_te, tx_te, w)
    
    loss_tr, loss_te = my.compute_classerror(w, tx_tr, tx_te, y_tr, y_te)
       
    return loss_tr, loss_te

def cross_validation_demo(y, X):
    # parameters
    seed = 56
    k_fold = 10
    
    # hyperparameters
    degrees = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    lambdas = np.logspace(-4, 1, 30)
    
    # split data in k fold
    k_indices = my.build_k_indices(y, k_fold, seed)
    
    # define lists to store the loss of training data and test data
    rmse_tr = np.zeros((len(degrees),k_fold))
    rmse_te = np.zeros((len(degrees),k_fold))
    best_lambda = np.zeros((len(degrees),))
    
    # Hyperparameter 1
    for d, degree in enumerate(degrees):
        print(degree)
        # Build polynomial function
        tX = my.build_poly(X, degree)
        
        loss_tr = np.zeros((len(lambdas),k_fold))
        loss_te = np.zeros((len(lambdas),k_fold))
        
        for l, lamb in enumerate(lambdas):
        
            # Cross-validation
            for k in range(k_fold):
                loss_tr[l,k], loss_te[l,k] = cross_validation(y, tX, k_indices, k, lamb)
                
        best_idx = np.argmin(np.mean(loss_te,axis=0))
        best_lambda[d] = lambdas[best_idx]
        
        rmse_tr[d,:] = loss_tr[best_idx,:]
        rmse_te[d,:] = loss_te[best_idx,:]
        
    bias_variance_decomposition_visualization(degrees, rmse_tr.T, rmse_te.T)   
    print(best_lambda)
    return rmse_tr, rmse_te, degrees, best_lambda

rmse_tr, rmse_te, degrees, lambdas = cross_validation_demo(y, tX)

## Make your predictions
First, train on the all datatset

In [ ]:
# Select the hyperparameter that have been chosen
final_degree = 10
final_lambda = 0.00239503

# Build the polynomial basis, perform ridge regression
final_X = my.build_poly(tX, final_degree)
final_weights = my.ridge_regression(y, final_X, final_lambda)

# Take a look at the train error
y_pred = predict_labels(final_weights, final_X)
loss = len(np.nonzero(y_pred-y)[0])/len(y)
loss

In [ ]:
# Download test data
DATA_TEST_PATH = 'test.csv'
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

# Transform test data as it has been done for training : cleaning and polynomial basis
tX_test, mean_xtest, std_xtest = clean_data(X_test)
final_X_test = my.build_poly(tX_test, final_degree)

# Aplly the model to the test data in order to get predictions
OUTPUT_PATH = 'results.csv'
y_pred = my.predict_labels(final_weights, final_X_test)
# create submission file
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

## Method 5 : Logistic regression

In [ ]:
def logic_cross_validation(y, x, k_indices, k):
    """return the loss of ridge regression."""
    gamma = 0.01
    w = np.zeros((x.shape[1],1))
    batch_size = 100
    max_epochs = 250
    
    # get k'th subgroup in test, others in train: TODO
    tx_tr = x[np.delete(k_indices, (k), axis=0).flatten()]
    y_tr = y[np.delete(k_indices, (k), axis=0).flatten()]
    tx_te = x[k_indices[k]]
    y_te = y[k_indices[k]]
    
    # Stochastics gradient descent
    losses, ws = my.learning_by_stochastic_newton_method(y_tr, tx_tr, w, batch_size, max_epochs, gamma)
    best_idx = np.argmin(losses)
    w = ws[best_idx]
    print(losses[best_idx])
    
    ## Full gradient descent leads to Memory Error, That's why we use stochastics GD
    
    loss_tr, loss_te = my.compute_logic_classerror(w, tx_tr, tx_te, y_tr, y_te)
    print(loss_tr, loss_te)
        
    return loss_tr, loss_te



def logistic_regression_newton_method_demo(y, tx):
    
    y[np.where(y == -1)[0]] = 0
    seed = 74
    
    # prepare k-fold cross-validation
    k_fold = 10
    k_indices = my.build_k_indices(y, k_fold, seed)
    
    # losses matrices
    loss_tr = np.zeros((k_fold,))
    loss_te = np.zeros((k_fold,))
    # cross-validation
    for k in range(k_fold):
        print(k)
        loss_tr[k], loss_te[k] = logic_cross_validation(y, tx, k_indices, k)
    
    loss_tr[k], loss_te[k] = logic_cross_validation(y, tx, k_indices, k)
    
    return loss_tr, loss_te


loss_tr, loss_te = logistic_regression_newton_method_demo(y, tX)

## Method 6 : Logistic penalized regression

In [ ]:
import myFunctions as my
from plots import cross_validation_visualization
def logic_cross_validation(y, x, k_indices, k, lambda_):
    """return the loss of ridge regression."""
    gamma = 0.01
    w = np.zeros((x.shape[1],1))
    batch_size = 5
    max_epochs = 300
    
    # get k'th subgroup in test, others in train: TODO
    tx_tr = x[np.delete(k_indices, (k), axis=0).flatten()]
    y_tr = y[np.delete(k_indices, (k), axis=0).flatten()]
    tx_te = x[k_indices[k]]
    y_te = y[k_indices[k]]
    
    # Stochastics gradient descent
    losses, ws = my.learning_by_stochastic_penalized_gradient_descent(y_tr, tx_tr, w, batch_size, max_epochs, gamma, lambda_)
    best_idx = np.argmin(losses)
    w = ws[best_idx]
    
    loss_tr, loss_te = my.compute_logic_classerror(w, tx_tr, tx_te, y_tr, y_te)
    print(loss_tr, loss_te)
        
    return loss_tr, loss_te



def logistic_regression_newton_method_demo(y, tx):
    
    y[np.where(y == -1)[0]] = 0
    lambdas = np.logspace(-5,-1,5)
    
    seed = 74
    
    # prepare k-fold cross-validation
    k_fold = 10
    k_indices = my.build_k_indices(y, k_fold, seed)
    
    #tx = my.build_poly(tx,2)
    
    # losses matrices
    loss_tr = np.zeros((k_fold,len(lambdas)))
    loss_te = np.zeros((k_fold,len(lambdas)))
    
    for l, lambda_ in enumerate(lambdas):
        print(l+1)
        
        # cross-validation
        for k in range(k_fold):
            loss_tr[k,l], loss_te[k,l] = logic_cross_validation(y, tx, k_indices, k, lambda_)
            
    mse_tr = np.mean(loss_tr, axis=0)
    mse_te = np.mean(loss_te, axis=0)
    cross_validation_visualization(lambdas, mse_tr, mse_te)   
    
    return loss_tr, loss_te


loss_tr, loss_te = logistic_regression_newton_method_demo(y, tX)



1


In [ ]:
gamma = 0.01
w = np.zeros((tX.shape[1],1))
batch_size = 10
max_epochs = 200
lambda_ = np.logspace(-5,-1,5)[3]
y[np.where(y == -1)[0]] = 0

losses, ws = my.learning_by_stochastic_penalized_newton_method(y, tX, w, batch_size, max_epochs, gamma, lambda_)
best_idx = np.argmin(losses)
w = ws[best_idx]
print(losses[best_idx])
    

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
tX_test, mean_xtest, std_xtest = clean_data(X_test)
#final_X_test = my.build_poly(tX_test, degree)

In [ ]:
OUTPUT_PATH = 'results.csv' # TODO: fill in desired name of output file for submission
y_pred = my.predict_logic_labels(w, tX_test)
y_pred[np.where(y_pred == 0)[0]] = -1
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)